<a href="https://colab.research.google.com/github/Padhul/SistemAntrianPengambilanBuku/blob/main/Python_Loket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio gTTS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


In [2]:
!pip install --upgrade gradio


In [3]:
!pip install gradio reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.8 MB/s eta 0:00:00


In [4]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=391961cd6371ed1ed3a00f68b6c4376876452d21501ab55a5cef8b5fb918e323
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [5]:
# sistem_antrian_dan_perpustakaan_no_returnfunc_no_bk_ui.py
# Versi: UI Pengguna TANPA fungsi return_loan dan TANPA Notifikasi BK di UI pengguna
# Python 3 - Gradio - Colab-friendly
import gradio as gr
import json, os, time
from datetime import date, datetime, timedelta

DATA_FILE = "data_antrian.json"
PDF_FOLDER = "pdf_tickets"
AUDIO_FOLDER = "audio_antrian"
os.makedirs(PDF_FOLDER, exist_ok=True)
os.makedirs(AUDIO_FOLDER, exist_ok=True)

# ---------------- DB helpers ----------------
def load_data():
    if not os.path.exists(DATA_FILE):
        data = {
            "nomor_terakhir": 0,
            "antrian": [],
            "daftar_waktu_layanan": [],
            "sedang_dilayani": {1: 0, 2: 0},
            "waktu_mulai": {1: None, 2: None},
            "records": [],
            "loans": [],
            "last_loan_id": 0,
            "bk_notifications": []
        }
        save_data(data)
        return data
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

def save_data(data):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ---------------- PDF helpers (reportlab optional) ----------------
def generate_ticket_pdf(nomor, nama, kelas, tanggal_str):
    try:
        from reportlab.lib.pagesizes import A6, landscape
        from reportlab.pdfgen import canvas
    except Exception:
        return None
    fname = f"ticket_{nomor}.pdf"
    path = os.path.join(PDF_FOLDER, fname)
    width, height = landscape(A6)
    c = canvas.Canvas(path, pagesize=(width, height))
    c.setFont("Helvetica-Bold", 16)
    c.drawCentredString(width/2, height - 30, "TIKET ANTRIAN")
    c.setFont("Helvetica-Bold", 36)
    c.drawCentredString(width/2, height - 70, f"#{nomor}")
    c.setFont("Helvetica", 10)
    c.drawCentredString(width/2, height - 100, f"Tanggal: {tanggal_str}")
    left_x = 18
    y = height - 140
    line_gap = 14
    c.setFont("Helvetica-Bold", 10)
    c.drawString(left_x, y, "Nama")
    c.setFont("Helvetica", 12)
    c.drawString(left_x + 60, y, nama)
    y -= line_gap
    c.setFont("Helvetica-Bold", 10)
    c.drawString(left_x, y, "Kelas")
    c.setFont("Helvetica", 12)
    c.drawString(left_x + 60, y, kelas)
    c.setFont("Helvetica-Oblique", 7)
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.drawRightString(width - 8, 8, f"Dicetak: {ts}")
    c.showPage()
    c.save()
    return path

def generate_user_loans_pdf(nama_peminjam, kelas_peminjam, loans_list, note_text):
    try:
        from reportlab.lib.pagesizes import A4
        from reportlab.pdfgen import canvas
        from reportlab.lib.units import mm
    except Exception:
        return None

    user_loans = [ln for ln in loans_list if (ln.get("nama","").strip().lower() == (nama_peminjam or "").strip().lower() and ln.get("kelas","").strip().lower() == (kelas_peminjam or "").strip().lower())]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"loans_{nama_peminjam.replace(' ','_')}_{timestamp}.pdf"
    path = os.path.join(PDF_FOLDER, fname)

    c = canvas.Canvas(path, pagesize=A4)
    width, height = A4
    margin = 20 * mm
    x = margin
    y = height - margin

    c.setFont("Helvetica-Bold", 14)
    c.drawString(x, y, "Daftar Pinjaman - Perpustakaan (Per User)")
    c.setFont("Helvetica", 10)
    c.drawRightString(width - margin, y, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    y -= 18

    c.setFont("Helvetica", 11)
    c.drawString(x, y, f"Nama: {nama_peminjam}")
    y -= 14
    c.drawString(x, y, f"Kelas: {kelas_peminjam}")
    y -= 18

    c.setFont("Helvetica-Bold", 10)
    c.drawString(x, y, "ID")
    c.drawString(x + 40, y, "Judul Buku")
    c.drawString(x + 340, y, "Pinjam")
    c.drawString(x + 430, y, "Tenggat")
    y -= 12
    c.setFont("Helvetica", 10)

    if not user_loans:
        c.drawString(x, y, "(Tidak ada pinjaman untuk user ini)")
        y -= 18
    else:
        for ln in user_loans:
            if y < margin + 60:
                c.showPage()
                y = height - margin
            id_str = str(ln.get("loan_id", ""))
            title_str = ln.get("book_title", "")[:45]
            borrow = ln.get("borrow_date", "")
            due = ln.get("due_date", "")
            c.drawString(x, y, id_str)
            c.drawString(x + 40, y, title_str)
            c.drawString(x + 340, y, borrow)
            c.drawString(x + 430, y, due)
            y -= 14

    y -= 10
    c.line(margin, y, width - margin, y)
    y -= 16

    max_len = 100
    s = note_text or ""
    note_lines = []
    while s:
        note_lines.append(s[:max_len])
        s = s[max_len:]
    c.setFont("Helvetica", 10)
    for nl in note_lines:
        if y < margin + 30:
            c.showPage()
            y = height - margin
        c.drawString(x, y, nl)
        y -= 14

    c.setFont("Helvetica-Oblique", 8)
    c.drawRightString(width - margin, margin - 6, "Generated by Sistem Antrian — SMAN 1 NGADILUWIH")
    c.save()
    return path

# ---------------- Loan logic ----------------
def create_loan(nama, kelas, book_title):
    db = load_data()
    db['last_loan_id'] = int(db.get('last_loan_id', 0)) + 1
    loan_id = db['last_loan_id']
    borrow_dt = date.today()
    due_dt = borrow_dt + timedelta(days=7)
    loan = {
        "loan_id": loan_id,
        "nama": nama,
        "kelas": kelas,
        "book_title": book_title,
        "borrow_date": borrow_dt.isoformat(),
        "due_date": due_dt.isoformat(),
        "status": "loaned",
        "returned_date": None,
        "overdue_days": 0
    }
    db.setdefault("loans", []).append(loan)
    save_data(db)
    msg = (f"✅ Buku dipinjam: '{book_title}'\n"
           f"ID Pinjam: {loan_id}\n"
           f"Tanggal pinjam: {borrow_dt.isoformat()}\n"
           f"Tenggat kembali: {due_dt.isoformat()} (7 hari)")
    return msg, loan_id, loan

# NOTE: fungsi return_loan telah DIHAPUS dari UI pengguna (tidak tersedia di sini).
# Pengembalian hanya dikelola di panel Operator (file operator terpisah).

def render_loans_html(db):
    loans = db.get("loans", [])
    if not loans:
        return "<div><i>Tidak ada data pinjaman.</i></div>"
    html = "<div style='font-family:system-ui;padding:8px'><table>"
    html += "<tr><th>ID</th><th>Nama</th><th>Kelas</th><th>Buku</th><th>Pinjam</th><th>Tenggat</th><th>Status</th></tr>"
    for ln in sorted(loans, key=lambda x: (x.get("status")!="loaned", x.get("loan_id"))):
        html += "<tr>"
        html += f"<td>{ln.get('loan_id')}</td>"
        html += f"<td>{ln.get('nama')}</td>"
        html += f"<td>{ln.get('kelas')}</td>"
        html += f"<td>{ln.get('book_title')}</td>"
        html += f"<td>{ln.get('borrow_date')}</td>"
        html += f"<td>{ln.get('due_date')}</td>"
        status_text = ln.get('status')
        if status_text == 'returned' and ln.get('overdue_days',0)>0:
            status_text += f" (telat {ln.get('overdue_days')} hari)"
        html += f"<td>{status_text}</td>"
        html += "</tr>"
    html += "</table></div>"
    return html

# ---------------- Nomor antrian & tiket logic (existing) ----------------
def ambil_nomor_dan_cetak(nama, kelas, tanggal):
    nama = (nama or "").strip()
    kelas = (kelas or "").strip()
    tanggal_input = (tanggal or "").strip()
    if not nama or not kelas:
        return "❌ Mohon isi Nama dan Kelas.", None
    tanggal_str = date.today().isoformat() if not tanggal_input else tanggal_input
    db = load_data()
    db["nomor_terakhir"] = int(db.get("nomor_terakhir", 0)) + 1
    nomor_baru = db["nomor_terakhir"]
    db.setdefault("antrian", []).append(nomor_baru)
    rec = {"nomor": nomor_baru, "nama": nama, "kelas": kelas, "tanggal": tanggal_str, "created_at": datetime.now().isoformat()}
    db.setdefault("records", []).append(rec)
    save_data(db)
    pdf_path = generate_ticket_pdf(nomor_baru, nama, kelas, tanggal_str)
    md = (f"### ✅ Nomor Antrian Anda: **{nomor_baru}**\n\n- Nama: **{nama}**\n- Kelas: **{kelas}**\n- Tanggal: **{tanggal_str}**\n\nTerlampir tiket (PDF) bila fungsi PDF tersedia.")
    if pdf_path:
        return md, pdf_path
    else:
        note = "\n\n⚠️ PDF tidak dibuat (library `reportlab` tidak ditemukan). Jika ingin PDF, jalankan `!pip install reportlab` lalu ulangi."
        return md + note, None

# ---------------- Gradio UI (Pengguna tanpa BK notifications) ----------------
with gr.Blocks(title="Sistem Antrian & Perpustakaan — SMAN 1 NGADILUWIH") as demo:
    gr.Markdown("# 🎉 Sistem Antrian & Perpustakaan")
    with gr.Row():
        with gr.Column():
            gr.Markdown("## Ambil Nomor / Ticket")
            nama_in = gr.Textbox(label="Nama", placeholder="Masukkan nama lengkap")
            kelas_in = gr.Textbox(label="Kelas", placeholder="Contoh: XI-IPA 1")
            tanggal_in = gr.Textbox(label="Tanggal (opsional, YYYY-MM-DD)", placeholder="kosong = hari ini")
            btn_ticket = gr.Button("🎟️ Ambil Nomor & Cetak (opsional PDF)")
            output_md = gr.Markdown("Hasil akan muncul di sini")
            ticket_file_out = gr.File(label="Unduh Tiket PDF (jika dibuat)")

            gr.Markdown("---")
            gr.Markdown("## Pinjam Buku")
            book_title_in = gr.Textbox(label="Judul Buku", placeholder="Masukkan judul buku yang ingin dipinjam")
            btn_pinjam = gr.Button("📚 Pinjam Buku (7 hari)")
            msg_pinjam = gr.Markdown("")
            loans_pdf_out = gr.File(label="Unduh PDF Daftar Pinjaman (per-user)")

            gr.Markdown("---")
            gr.Markdown("**Catatan:** Pengguna tidak dapat mengembalikan buku lewat halaman ini.\nPengembalian dikelola oleh petugas/operator melalui panel operator.")
        with gr.Column():
            gr.Markdown("### Daftar Semua Pinjaman (admin/operator lihat seluruhnya)")
            loans_html = gr.HTML(render_loans_html(load_data()))
            # Notifikasi BK sengaja tidak ditampilkan di UI pengguna

    # Bindings
    btn_ticket.click(fn=ambil_nomor_dan_cetak, inputs=[nama_in, kelas_in, tanggal_in], outputs=[output_md, ticket_file_out])

    def handle_pinjam(nama, kelas, book_title):
        if not nama or not kelas:
            return "❌ Mohon isi Nama dan Kelas sebelum meminjam.", render_loans_html(load_data()), None
        if not book_title or not book_title.strip():
            return "❌ Mohon isi Judul Buku yang akan dipinjam.", render_loans_html(load_data()), None
        msg, loan_id, loan = create_loan(nama.strip(), kelas.strip(), book_title.strip())
        note_text = (f"✅ Buku dipinjam: '{loan['book_title']}' ID Pinjam: {loan_id} "
                     f"Tanggal pinjam: {loan['borrow_date']} Tenggat kembali: {loan['due_date']} (7 hari)")
        db = load_data()
        pdf_path = generate_user_loans_pdf(nama.strip(), kelas.strip(), db.get("loans", []), note_text)
        return msg, render_loans_html(db), pdf_path

    btn_pinjam.click(fn=handle_pinjam, inputs=[nama_in, kelas_in, book_title_in], outputs=[msg_pinjam, loans_html, loans_pdf_out])

# launch
if __name__ == "__main__":
    demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b6eac1901dbf434774.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b6eac1901dbf434774.gradio.live


In [ ]:
# operator_panel_no_bk_ui.py
# Operator — Panggil Loket & Audio (BK notifications HIDDEN)
import gradio as gr
import time, os, json
from collections import deque
from gtts import gTTS

DATA_FILE = "data_antrian.json"
AUDIO_FOLDER = "audio_antrian"
os.makedirs(AUDIO_FOLDER, exist_ok=True)

# ---------- DB helpers ----------
def load_db():
    if not os.path.exists(DATA_FILE):
        db = {
            "nomor_terakhir": 0,
            "antrian": [],
            "daftar_waktu_layanan": [],
            "sedang_dilayani": {1:0, 2:0},
            "waktu_mulai": {1: None, 2: None},
            "loans": [],
            "last_loan_id": 0,
            "bk_notifications": []
        }
        save_db(db)
        return db
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        raw = json.load(f)
    db = {
        "nomor_terakhir": raw.get("nomor_terakhir", 0),
        "antrian": raw.get("antrian", []),
        "daftar_waktu_layanan": raw.get("daftar_waktu_layanan", []),
        "sedang_dilayani": {int(k): v for k,v in raw.get("sedang_dilayani", {"1":0,"2":0}).items()},
        "waktu_mulai": {int(k): v for k,v in raw.get("waktu_mulai", {"1":None,"2":None}).items()},
        "loans": raw.get("loans", []),
        "last_loan_id": raw.get("last_loan_id", 0),
        "bk_notifications": raw.get("bk_notifications", [])
    }
    return db

def save_db(db):
    to_save = {
        "nomor_terakhir": db.get("nomor_terakhir", 0),
        "antrian": list(db.get("antrian", [])),
        "daftar_waktu_layanan": db.get("daftar_waktu_layanan", []),
        "sedang_dilayani": {str(k): v for k, v in db.get("sedang_dilayani", {1:0,2:0}).items()},
        "waktu_mulai": {str(k): v for k, v in db.get("waktu_mulai", {1:None,2:None}).items()},
        "loans": db.get("loans", []),
        "last_loan_id": db.get("last_loan_id", 0),
        "bk_notifications": db.get("bk_notifications", [])
    }
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(to_save, f, ensure_ascii=False, indent=2)

# ---------- Utilities ----------
def format_waktu(d):
    if d is None:
        return "-"
    if d < 60:
        return f"{int(d)} detik"
    m = int(d//60); s = int(d%60)
    return f"{m} menit {s} detik"

def hitung_rata(arr):
    return sum(arr)/len(arr) if arr else 0

def render_queue_html(antrian):
    if not antrian:
        return "<div><i>Tidak ada antrian</i></div>"
    s = "<ol>"
    for i,n in enumerate(antrian, start=1):
        s += f"<li><b>#{n}</b> (pos {i})</li>"
    s += "</ol>"
    return s

def render_stats_html(db):
    arr = db.get("daftar_waktu_layanan", [])
    total = len(arr)
    rata = hitung_rata(arr)
    fastest = min(arr) if arr else 0
    slowest = max(arr) if arr else 0
    html = "<div>"
    html += f"<div><b>Terlayani:</b> {total}</div>"
    html += f"<div><b>Rata-rata:</b> {format_waktu(rata) if total>0 else '-'}</div>"
    html += "<div style='margin-top:8px'><b>Loket:</b></div>"
    for lok in [1,2]:
        st = db["sedang_dilayani"].get(lok, 0)
        html += f"<div>Loket {lok}: {('Nomor '+str(st)) if st!=0 else '<i>kosong</i>'}</div>"
    html += f"<div style='margin-top:6px; color:gray; font-size:smaller;'>Tercepat: {format_waktu(fastest) if total>0 else '-'} — Terlama: {format_waktu(slowest) if total>0 else '-'}</div>"
    html += "</div>"
    return html

# ---------- TTS ----------
def make_tts(nomor, loket):
    fn = f"nomor_{nomor}_loket_{loket}.mp3"
    path = os.path.join(AUDIO_FOLDER, fn)
    if not os.path.exists(path):
        try:
            tts = gTTS(text=f"Nomor {nomor}, silakan menuju loket {loket}.", lang='id')
            tts.save(path)
        except Exception as e:
            print("TTS error:", e)
            return None
    return os.path.abspath(path)

# ---------- Operator actions ----------
def op_status():
    db = load_db()
    info = f"Antrian: {db['antrian'] if db['antrian'] else 'kosong'}\nSedang dilayani: Loket1={db['sedang_dilayani'].get(1,0)}, Loket2={db['sedang_dilayani'].get(2,0)}"
    return info, gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db))

def op_call(loket):
    db = load_db()
    lok = int(loket)
    if db["sedang_dilayani"].get(lok,0) != 0:
        return f"Loket {lok} sedang melayani nomor {db['sedang_dilayani'][lok]}", None, gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db))
    if not db["antrian"]:
        return "Antrian kosong", None, gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db))
    nomor = db["antrian"].pop(0)
    db["sedang_dilayani"][lok] = nomor
    db["waktu_mulai"][lok] = time.time()
    save_db(db)
    audio_abs = make_tts(nomor, lok)
    msg = f"NOMOR {nomor}, silakan menuju Loket {lok}!"
    # fallback HTML if needed
    audio_html = f'<audio controls autoplay src="file://{audio_abs}"></audio>' if audio_abs else ""
    return msg, audio_abs, gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db)), gr.HTML(audio_html)

def op_complete(loket):
    db = load_db()
    lok = int(loket)
    cur = db["sedang_dilayani"].get(lok,0)
    if cur == 0:
        return f"Tidak ada yang dilayani di Loket {lok}", gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db))
    start = db["waktu_mulai"].get(lok)
    dur = (time.time() - start) if start else 0
    db["daftar_waktu_layanan"].append(dur)
    db["sedang_dilayani"][lok] = 0
    db["waktu_mulai"][lok] = None
    save_db(db)
    return f"Selesai: Nomor {cur}. Waktu: {format_waktu(dur)}", gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db))

def op_reset_execute():
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    db = {
        "nomor_terakhir": 0,
        "antrian": [],
        "daftar_waktu_layanan": [],
        "sedang_dilayani": {1:0,2:0},
        "waktu_mulai": {1: None, 2: None},
        "loans": [],
        "last_loan_id": 0,
        "bk_notifications": []
    }
    save_db(db)
    return "Data berhasil direset!", gr.HTML(render_queue_html(db["antrian"])), gr.HTML(render_stats_html(db))

# ---------- Gradio UI (Operator) ----------
with gr.Blocks(title="Operator — Panggil & Audio (No BK visible)") as demo:
    gr.Markdown("## Operator Panel — Panggil Loket & Audio")
    with gr.Row():
        with gr.Column(scale=1):
            btn_call1 = gr.Button("🔊 Panggil Loket 1")
            btn_done1 = gr.Button("✅ Selesai Loket 1")
            btn_call2 = gr.Button("🔊 Panggil Loket 2")
            btn_done2 = gr.Button("✅ Selesai Loket 2")
            btn_refresh = gr.Button("🔁 Refresh")
            btn_reset = gr.Button("⚠️ Reset Data")
        with gr.Column(scale=2):
            status_out = gr.Textbox(interactive=False, label="Status")
            audio_out = gr.Audio(label="Audio Panggilan", autoplay=True)
            queue_html = gr.HTML("<i>Memuat antrian...</i>")
            stats_html = gr.HTML("<i>Memuat statistik...</i>")

    # load initial UI state
    demo.load(fn=op_status, outputs=[status_out, queue_html, stats_html])

    # Bindings (note outputs mapping must match return order)
    btn_refresh.click(fn=op_status, outputs=[status_out, queue_html, stats_html])
    # op_call returns: msg, audio_path, queue_html, stats_html, audio_fallback_html
    btn_call1.click(fn=lambda: op_call(1), outputs=[status_out, audio_out, queue_html, stats_html, gr.HTML()])
    btn_call2.click(fn=lambda: op_call(2), outputs=[status_out, audio_out, queue_html, stats_html, gr.HTML()])
    btn_done1.click(fn=lambda: op_complete(1), outputs=[status_out, queue_html, stats_html])
    btn_done2.click(fn=lambda: op_complete(2), outputs=[status_out, queue_html, stats_html])
    btn_reset.click(fn=op_reset_execute, outputs=[status_out, queue_html, stats_html])

if __name__ == "__main__":
    demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c641cbbcc4477d200d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
